In [17]:

#Import various needed dictionaries
import numpy as np
import datetime as dt
import string
from collections import deque

#This function asks the user for their name and returns it, as well as the current date.
def header():
    #name = input("Vor-und Nachname:")
    name='Matthew Adams'
    date = dt.date.today()
    return name, date

#Create a generic array of abc's, convert it to numbers, and return it
def normal_list():
    return list(range(1,27))

#return a standard alphabet list
def standard_alphabet():
    alphabet_string = string.ascii_lowercase
    alphabet_list = list(alphabet_string)
    return alphabet_list

#create a rotor by assembling a standard with a randomized alphabet list
def create_rotor(translation):
    return dict(zip(normal_list(),translation))

#create an array of historically accurate rotors using the create_rotor function
def create_rotors(rotor_num, rotation):
    if rotor_num=='Beta':
        rotor_num=9
    elif rotor_num=='Gamma':
        rotor_num=10
    if rotor_num=='Reflector B':
        rotor_num=11
    if rotor_num=='Reflector C':
        rotor_num=12
    rotor=[]
    rotor.append(create_rotor(''))
    #rotor 1
    rotor.append('ekmflgdqvzntowyhxuspaibrcj')
    #rotor 2
    rotor.append('ajdksiruxblhwtmcqgznpyfvoe')
    #rotor 3
    rotor.append('bdfhjlcprtxvznyeiwgakmusqo')
    #rotor 4
    rotor.append('esovpzjayquirhxlnftgkdcmwb')
    #rotor 5
    rotor.append('vzbrgityupsdnhlxawmjqofeck')
    #rotor 6
    rotor.append('jpgvoumfyqbenhzrdkasxlictw')
    #rotor 7
    rotor.append('nzjhgrcxmyswboufaivlpekqdt')
    #rotor 8
    rotor.append('fkqhtlxocbjspdzramewniuygv')
    #Beta Wheel
    rotor.append('leyjvcnixwpbqmdrtakzgfuhos')
    #Gamma Wheel
    rotor.append('fsokanuerhmbtiycwlqpzxvgjd')
    #Reflector B Thin
    rotor.append('enkqauywjicopblmdxzvfthrgs')
    #Reflector C Thin
    rotor.append('rdobjntkvehmlfcwzaxgyipsuq')
    numbers = np.array([])
    selected_rotor=rotor[rotor_num]
    for letter in selected_rotor:
        number = ord(letter) - 96
        numbers = np.append(numbers,int(number))
    #create the 'offset' of rotor8 by comparing rotor8 to the abc array and print it
    difference = np.subtract(numbers,normal_list())
    #**difference is the offset that is needed** next, we can just rotate it using the wheel setting.
    new = deque(difference)
    new.rotate(rotation)
    offset= dict(zip(range(0,26),new))
    return offset

#This is used to construct rotors that will be used to translate the values as they move
#back through the machine after the reflector.
def rotor_invert(rotor):
    keys=np.array(list(rotor.keys()))
    values=np.array(list(rotor.values()))
    difference = np.add(keys,values)
    difference=(difference+26)%26
    values=values*-1
    invert_rotor= dict(zip(difference,values))
    return invert_rotor

#This is the plug board used in the default code I'll be decripting
def default_plug_board():
    plugs='efmqabguinkxcjordpzthwvlys'    
    return dict(zip(standard_alphabet(),plugs))

#This is the main portion of the code. it creates a header, plug_board, 4 rotors and the reflector
#It then encodes the message.
def main():
    name, date = header()
    plug_board = default_plug_board()
    rotor8 = create_rotors(8,11)
    rotor6 = create_rotors(6,4)
    rotor5 = create_rotors(5,15)
    beta_wheel = create_rotors('Beta',4)
    reflector = create_rotors('Reflector C',0)
    beta_wheel_II = rotor_invert(beta_wheel)
    rotor5_II = rotor_invert(rotor5)
    rotor6_II = rotor_invert(rotor6)
    rotor8_II = rotor_invert(rotor8)

    message_wspaces='lanotctouarbbfpmhphgczxtdygahgufxgewkblkgjwlqxxtgpjjavtockzfslppqihzfxoebwiiekfzlcloaqjuljoyhssmbbgwhzanvoiipyrbrtdjqdjjoqkcxwdnbbtyvxlytapgveatxsonpnynqfudbbhhvwepyeydohnlxkzdnwrhduwujumwwviiwzxiviuqdrhymncyefuapnhotkhkgdnpsaknuaghjzsmjbmhvtreqedgxhlzwifuskdqvelnmimithbhdbwvhdfyhjoqihortdjdbwxemeayxgyqxohfdmyuxxnojazrsghplwmlrecwwutlrttvlbhyoorglgowuxnxhmhyfaacqekthsjw'    
    final_message = np.array([])
    # This section of the code simulates the rotors spinning. It begins with the rotors
    # set in the correct positions and then it rotates them identical to the actual machine.
    # Think of the rotors as the second, minute, and hour hand of a watch, the one furthest
    # to the right rotates the most quickly and the others rotate more slowly. The rotation
    # of one rotor is based on the movement of the rotor to it's right.
    a=25 #%%%FOR NOW.. THIS WILL CAUSE PROBLEMS LATER %%%
    b=18
    c=3
    d=2
    for letter in message_wspaces:
        if b==25:
            b=0
            c=(c+1)%25
        if b==12:
            b=13
            c=(c+1)%25
        if a==12 or a==25:
            a+=1
            a=a%26
            if b==25 or b==12:
                b+=1
                b=b%26
                c+=1
                c=c%26
            else:
                b+=1
        else:
            a+=1
            
        # This is the process of encrypting the letter, starting with the plugboard, then the rotors,
        # beta wheel, and reflector. Then the letter comes through the machine in reverse.
        #That is why there are two translation blocks below.
        after_plugboard=plug_board[letter]
        after_plugboard_number=ord(after_plugboard) - 97
        after_plugboard_number=(after_plugboard_number+a)%26
        after_first_rotor = int((after_plugboard_number+rotor8[after_plugboard_number])%26)+97
        after_first_rotor=(after_first_rotor+b-a-97)%26 
        after_second_rotor = int((after_first_rotor+rotor6[after_first_rotor])%26)+97
        after_second_rotor=(after_second_rotor+c-b-97)%26
        after_third_rotor = int((after_second_rotor+rotor5[after_second_rotor])%26)+97
        after_third_rotor=(after_third_rotor+d-c-97)%26 
        after_beta_wheel = int((after_third_rotor+beta_wheel[after_third_rotor])%26)+97 
        after_beta_wheel=(after_beta_wheel-d-97)%26
        after_reflector = int((after_beta_wheel+reflector[after_beta_wheel])%26)+97
        after_reflector=(after_reflector+d-97)%26
        
        ##Coming back through the machine now
        after_beta_wheel_II = int((after_reflector+beta_wheel_II[after_reflector])%26)+97
        after_beta_wheel_II=(after_beta_wheel_II-d+c-97)%26
        after_third_rotor_II = int((after_beta_wheel_II+rotor5_II[after_beta_wheel_II])%26)+97
        after_third_rotor_II=(after_third_rotor_II-c+b-97)%26
        after_second_rotor_II = int((after_third_rotor_II+rotor6_II[after_third_rotor_II])%26)+97
        after_second_rotor_II = (after_second_rotor_II-b+a-97)%26
        after_first_rotor_II = int((after_second_rotor_II+rotor8_II[after_second_rotor_II])%26)+97
        after_first_rotor_II = (after_first_rotor_II-a-97+26)%26
        final=plug_board[chr(after_first_rotor_II+97)]
        final_message = np.append(final_message,final)
        
#         ## This section is useful for debugging, it prints out the letters after each level
#         ## of encryption
#         print(letter,after_plugboard,chr(after_first_rotor+97),chr(after_second_rotor+97)\
#               ,chr(after_third_rotor+97),chr(after_beta_wheel+97),chr(after_reflector+97)\
#               ,chr(after_beta_wheel_II+97),chr( after_third_rotor_II+97),chr(after_second_rotor_II+97)\
#               ,chr(after_first_rotor_II+97), final,d,c,b,a)
    return(final_message)
final_message=main()

#This portion of the code is where we test to see if the result of the machine is
# accurate against the ans
answer_key = list('krkrallexxfolgendesistsofortbekanntzugebenxxichhabefolgelnbebefehlerhaltenxxjansterledesbisherigxnreichsmarschallsjgoeringjsetztderfuehrersieyhvrrgrzssadmiralyalsseinennachfolgereinxschriftlschevollmachtunterwegsxabsofortsollensiesaemtlichemassnahmenverfuegenydiesichausdergegenwaertigenlageergebenxgezxreichsleiteikktulpekkjbormannjxxobxdxmmmdurnhfkstxkomxadmxuuubooiexkp')
#print(len(answer_key))
R=0
success_ratio = np.array([])
for letter in final_message:
    #print(final_message[R]==answer_key[R], final_message[R],answer_key[R])
    success_ratio=np.append(success_ratio, final_message[R]==answer_key[R])
    R+=1
    
print('The machine successfully decrypted ', int(sum(success_ratio)), ' out of ', len(success_ratio), ' characters correctly, an accuracy rating of ',sum(success_ratio)*100/len(success_ratio), '%')



The machine successfully decrypted  372  out of  372  characters correctly, an accuracy rating of  100.0 %


In [23]:
from pathlib import Path
# This portion of the code translates the message from German to English using a 
# Google Translate API. In order for this to work, I had to set up a Google Cloud account,
# which costs money per run. As such, I didn't want to post my credentials online,
# and this section of code will not run without it. Let me know if you'd like to see a demonstration
path=Path("C://Users//Matt//Documents//Enigma//blissful-hash-317522-0e8fcafeb67a.json")
GOOGLE_APPLICATION_CREDENTIALS=str(path)
print(GOOGLE_APPLICATION_CREDENTIALS)
from google.cloud import translate_v2 as translate
translator=translate.Client.from_service_account_json(GOOGLE_APPLICATION_CREDENTIALS)
zebra = translator.translate('KRIEGSNOTMELDUNG An Alle:\
   Folgendes ist sofort bekanntzugeben:\
   Ich habe folgende Befehl erhalten:\
   Anstelle des bisherigen Reichsmarschalls Göring setzt der Führer\
   Sie, Herr Großadmiral, als seinen Nachfolger ein.\
   Schriftlische Vollmacht unterwegs.\
   Ab sofort sollen Sie sämtliche Maßnahmen verfügen,\
   die sich aus die gegenwärtigen Lage ergeben.\
   Gez. Reichsleiter (Tulpe) Bormann\
   Von Oberbefehlshaber der Marine,\
   durch Funkstelle der Kommandierender Admiral der Unterseeboote.')
print(zebra)

C:\Users\Matt\Documents\blissful-hash-317522-0e8fcafeb67a.json
{'translatedText': 'WAR NOTIFICATION To all: The following must be announced immediately: I have received the following order: Instead of the previous Reichsmarschall Göring, the Führer appoints you, Grand Admiral, as his successor. Written power of attorney on the way. From now on you should have all the measures that arise from the current situation. Signed Reichsleiter (Tulpe) Bormann From Commander-in-Chief of the Navy, through the radio station the commanding admiral of the submarines.', 'detectedSourceLanguage': 'de', 'input': 'KRIEGSNOTMELDUNG An Alle:   Folgendes ist sofort bekanntzugeben:   Ich habe folgende Befehl erhalten:   Anstelle des bisherigen Reichsmarschalls Göring setzt der Führer   Sie, Herr Großadmiral, als seinen Nachfolger ein.   Schriftlische Vollmacht unterwegs.   Ab sofort sollen Sie sämtliche Maßnahmen verfügen,   die sich aus die gegenwärtigen Lage ergeben.   Gez. Reichsleiter (Tulpe) Bormann   V